## Run this first if you want NDVI/NDWI basemaps

In [ ]:
import ee
ee.Authenticate()

## Vibes

In [1]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(filename='geovibes.log', level=logging.ERROR)

try:
    from src.ui import GeoVibes
    vibes = GeoVibes.from_config('config/java_config.json')
except Exception as e:
    logging.error(f"Jupyter kernel crashed: {str(e)}", exc_info=True)
    raise

✅ Earth Engine initialized with user credentials


HTML(value='\n        <style>\n        .widget-toggle-buttons button:nth-child(1).mod-active {\n            ba…

In [ ]:
import numpy as np

print("Query vector:", vibes.query_vector)
print("Query vector shape:", vibes.query_vector.shape if vibes.query_vector is not None else "None")
print("Query vector has NaN:", np.isnan(vibes.query_vector).any() if vibes.query_vector is not None else "N/A")

In [ ]:
# Check if your positive/negative embeddings contain NULLs
print("Positive IDs:", vibes.pos_ids)
print("Negative IDs:", vibes.neg_ids)

# Check cached embeddings
for pid in vibes.pos_ids[:3]:  # Check first few
    if pid in vibes.cached_embeddings:
        emb = vibes.cached_embeddings[pid]
        print(f"Embedding {pid}: shape={emb.shape}, has_nan={np.isnan(emb).any()}")
    else:
        print(f"Embedding {pid}: NOT CACHED")

In [ ]:
query = """
SELECT id, embedding 
FROM geo_embeddings 
WHERE embedding IS NULL 
"""
null_embeddings = vibes.duckdb_connection.execute(query).fetchall()
print("Embeddings with NULL:", len(null_embeddings))

# Check a specific positive point's embedding in DB
if vibes.pos_ids:
    pid = vibes.pos_ids[0]
    query = f"SELECT id, embedding FROM geo_embeddings WHERE id = '{pid}'"
    result = vibes.duckdb_connection.execute(query).fetchone()
    if result:
        emb = np.array(result[1])
        print(f"DB embedding for {pid}: shape={emb.shape}, has_nan={np.isnan(emb).any()}")

In [ ]:
# Approach 1: Try to use list functions to detect NULLs
try:
    null_check1 = vibes.duckdb_connection.execute("""
        SELECT COUNT(*) as count_with_nulls
        FROM geo_embeddings 
        WHERE embedding IS NOT NULL
        AND list_contains(embedding, NULL)
    """).fetchone()
    print("Method 1 - list_contains(embedding, NULL):", null_check1[0])
except Exception as e:
    print("Method 1 failed:", e)

# Approach 2: Check if any specific array positions contain NULL
try:
    null_check2 = vibes.duckdb_connection.execute("""
        SELECT COUNT(*) as count_with_nulls
        FROM geo_embeddings 
        WHERE embedding IS NOT NULL
        AND (
            embedding[1] IS NULL OR embedding[2] IS NULL OR embedding[3] IS NULL OR
            embedding[32] IS NULL OR embedding[64] IS NULL
        )
    """).fetchone()
    print("Method 2 - checking specific positions:", null_check2[0])
except Exception as e:
    print("Method 2 failed:", e)

# Approach 3: Check array length vs expected length
try:
    length_check = vibes.duckdb_connection.execute("""
        SELECT array_length(embedding) as len, COUNT(*) as count
        FROM geo_embeddings 
        WHERE embedding IS NOT NULL
        GROUP BY array_length(embedding)
        ORDER BY count DESC
    """).fetchall()
    print("Method 3 - Array lengths:")
    for length, count in length_check:
        print(f"  Length {length}: {count} embeddings")
except Exception as e:
    print("Method 3 failed:", e)

# Approach 4: Try to convert to string and look for 'null'
try:
    string_check = vibes.duckdb_connection.execute("""
        SELECT COUNT(*) as count_with_string_null
        FROM geo_embeddings 
        WHERE embedding IS NOT NULL
        AND LOWER(CAST(embedding AS VARCHAR)) LIKE '%null%'
        LIMIT 10
    """).fetchone()
    print("Method 4 - string contains 'null':", string_check[0])
except Exception as e:
    print("Method 4 failed:", e)